# Import thư viện

In [48]:
import numpy as np
import pandas as pd
from apyori import apriori

In [49]:
from collections import namedtuple
from itertools import combinations
from itertools import chain

# Đọc dữ liệu 

In [50]:
df=pd.read_csv(r"data.csv",header=None)

In [51]:
df.values

array([['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
       ['Wine', nan, 'Bread', 'Butter', 'Milk', nan],
       [nan, nan, 'Bread', 'Butter', 'Milk', nan],
       [nan, 'Chips', nan, nan, nan, 'Apple'],
       ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
       ['Wine', 'Chips', nan, nan, 'Milk', nan],
       ['Wine', 'Chips', 'Bread', 'Butter', nan, 'Apple'],
       ['Wine', 'Chips', nan, nan, 'Milk', nan],
       ['Wine', nan, 'Bread', nan, nan, 'Apple'],
       ['Wine', nan, 'Bread', 'Butter', 'Milk', nan],
       [nan, 'Chips', 'Bread', 'Butter', nan, 'Apple'],
       ['Wine', nan, nan, 'Butter', 'Milk', 'Apple'],
       ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', nan],
       ['Wine', nan, 'Bread', nan, 'Milk', 'Apple'],
       ['Wine', nan, 'Bread', 'Butter', 'Milk', 'Apple'],
       ['Wine', 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
       [nan, 'Chips', 'Bread', 'Butter', 'Milk', 'Apple'],
       [nan, 'Chips', nan, 'Butter', 'Milk', 'Apple'],
       [

# Chuyển dataframe dữ liệu thành dạng list

In [52]:
records=[]
for i in range(0,22):
    records.append([str(df.values[i,j]) for j  in range(0,6)])

# Thuật toán apriori từ thư viện apyori

Các công thức :

$supp(X)=\frac{\text{count}(X)}{\left|\mathcal{D}\right|}$

$conf(X \rightarrow Y) = \frac{\text{supp}(X,Y)}{\text{supp}(X)}$

$lift(X \rightarrow Y) = \frac{\text{conf}(X \rightarrow Y)}{\text{supp}(Y)}$

Một tập itemset chỉ có 1 giá trị support, tuy nhiên confidence và lift của các tập con cùng tạo ra tập đó có thể có các confidence và lift khác nhau

Ví dụ :

$\text{conf}(\{A,B\} \rightarrow C) \neq \text{conf}(\{B,C\} \rightarrow A), \text{lift}(\{A,B\} \rightarrow C) \neq \text{lift}(\{B,C\} \rightarrow A)$



In [53]:
association_rules=apriori(records,min_support=0.5,min_confidence=0.7,min_lift=1.2)

# Vì kết quả trả về là một generator object nên convert sang list để in ra

In [54]:
association_results=list(association_rules)

# Kết quả thô (raw values) 

In [55]:
association_results

[RelationRecord(items=frozenset({'Butter', 'Milk', 'Bread'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Butter'}), items_add=frozenset({'Milk', 'Bread'}), confidence=0.7333333333333334, lift=1.241025641025641), OrderedStatistic(items_base=frozenset({'Milk', 'Bread'}), items_add=frozenset({'Butter'}), confidence=0.8461538461538461, lift=1.241025641025641)])]

# Hàm chuyển đổi kết quả thô (raw values) sang dạng dataframe để dễ đọc (readable)

In [56]:
# Hàm inspect gốc (trong file pdf)
def inspect(output):
    lhs=[tuple(result[2][0][0])[0] for result in output]
    rhs=[tuple(result[2][0][1])[0] for result in output]
    support=[result[1] for result in output]
    confidence=[result[2][0][2] for result in output]
    lift=[result[2][0][3] for result in output]
    return list(zip(lhs,rhs,support,confidence,lift))


Kết quả dataframe của hàm inspect gốc

In [57]:
output_DataFrame=pd.DataFrame(inspect(association_results),columns=['Left_Hand_Side','Right_Hand_Side','Support','Confidence','Lift'])
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,Butter,Milk,0.5,0.733333,1.241026


Vì hàm inspect trong file pdf bị sai , in ra kết quả thiếu thông tin so với kết quả thô của hàm apriori nên em sẽ sửa lại hàm inspect để có thể in ra chính xác kết quả trả về của hàm apriori

In [58]:
# Hàm inspect sau khi chỉnh sửa
def edited_inspect(output):
    lhs=[]
    rhs=[]
    support=[]
    lift=[]
    confidence=[]
    for result in output:
        for ordered_statistic in result.ordered_statistics:
            support.append(result.support)
            lhs.append(ordered_statistic.items_base)
            rhs.append(ordered_statistic.items_add)
            confidence.append(ordered_statistic.confidence)
            lift.append(ordered_statistic.lift)
    return list(zip(lhs,rhs,support,confidence,lift))


Kết quả dataframe của hàm inspect sau khi chỉnh sửa 

In [59]:
output_DataFrame=pd.DataFrame(edited_inspect(association_results),columns=['Left_Hand_Side','Right_Hand_Side','Support','Confidence','Lift'])
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,(Butter),"(Milk, Bread)",0.5,0.733333,1.241026
1,"(Milk, Bread)",(Butter),0.5,0.846154,1.241026


Vì trường hợp min_confidence = 0.5 chỉ có một kết quả nên không cho thấy sự khác biệt của hàm inspect(hàm inspect gốc) và hàm edited_inspect(hàm inspect đã qua chỉnh sửa), nên em sẽ hạ mức min_confidence thấp xuống một chút(0,4) để so sánh kết quả giữa 2 hàm 

In [60]:
association_rules=apriori(records,min_support=0.4,min_confidence=0.7,min_lift=1.2)
association_results=list(association_rules)
association_results

[RelationRecord(items=frozenset({'Apple', 'Wine', 'Bread'}), support=0.45454545454545453, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Apple', 'Wine'}), items_add=frozenset({'Bread'}), confidence=0.9090909090909091, lift=1.25)]),
 RelationRecord(items=frozenset({'Apple', 'Milk', 'Butter'}), support=0.4090909090909091, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Apple', 'Milk'}), items_add=frozenset({'Butter'}), confidence=0.8181818181818182, lift=1.2000000000000002)]),
 RelationRecord(items=frozenset({'Butter', 'Milk', 'Bread'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Butter'}), items_add=frozenset({'Milk', 'Bread'}), confidence=0.7333333333333334, lift=1.241025641025641), OrderedStatistic(items_base=frozenset({'Milk', 'Bread'}), items_add=frozenset({'Butter'}), confidence=0.8461538461538461, lift=1.241025641025641)]),
 RelationRecord(items=frozenset({'Butter', 'Wine', 'Bread'}), support=0.45454545454545453, ordered_stati

In [61]:
output_DataFrame=pd.DataFrame(inspect(association_results),columns=['Left_Hand_Side','Right_Hand_Side','Support','Confidence','Lift'])
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,Apple,Bread,0.454545,0.909091,1.250000
1,Apple,Butter,0.409091,0.818182,1.200000
2,Butter,Milk,0.500000,0.733333,1.241026
3,Wine,Bread,0.454545,0.909091,1.250000
4,Wine,Milk,0.409091,0.818182,1.384615


In [62]:
output_DataFrame=pd.DataFrame(edited_inspect(association_results),columns=['Left_Hand_Side','Right_Hand_Side','Support','Confidence','Lift'])
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,"(Apple, Wine)",(Bread),0.454545,0.909091,1.250000
1,"(Apple, Milk)",(Butter),0.409091,0.818182,1.200000
2,(Butter),"(Milk, Bread)",0.500000,0.733333,1.241026
3,"(Milk, Bread)",(Butter),0.500000,0.846154,1.241026
4,"(Wine, Butter)",(Bread),0.454545,0.909091,1.250000
5,"(Wine, Butter)","(Milk, Bread)",0.409091,0.818182,1.384615
6,"(Milk, Wine, Bread)",(Butter),0.409091,0.818182,1.200000
7,"(Milk, Wine, Butter)",(Bread),0.409091,0.900000,1.237500


# Hàm apriori tự viết

namedtuple cũng giống như tuple bình thường nhưng không những cho phép ta truy cập theo chỉ số (index) mà còn theo tên thuộc tính

In [63]:
SupportRecord = namedtuple(
    'SupportRecord', ('items', 'support'))  # định nghĩa một bản ghi (record) SupportRecord bao gồm các trường items và support
RelationRecord = namedtuple(
    'RelationRecord', SupportRecord._fields + ('ordered_statistics',))  # định nghĩa một bản ghi (record) RelationRecord kế thừa các trường từ SupportRecord và thêm trường ordered_statistics
OrderedStatistic = namedtuple(
    'OrderedStatistic', ('items_base', 'items_add', 'confidence', 'lift',))  # định nghĩa một bản ghi (record) OrderedStatistic bao gồm các trường items_base, items_add, confidence và lift


# Hàm tính số lượng giao dịch, số lượng item và chỉ số index của chúng trong các giao dịch

In [64]:
def calc_num_items_index(transactions):
    num_transaction = 0  # Tổng số giao dịch
    items = []  # Danh sách các item
    items_index = {}  # Chỉ số index của item trong danh sách giao dịch
    for transaction in transactions:
        for item in transaction:
            if item not in items_index:
                items.append(item)
                items_index[item] = set()
            items_index[item].add(num_transaction)
        num_transaction += 1
    return num_transaction, items, items_index


# Hàm tính support của một tập itemset trong các giao dịch

In [65]:

def calc_support(transaction_index, num_transaction, itemset):
    if not itemset:
        return 1.0  # Nếu không có item, support = 1.0
    if not num_transaction:
        return 0.0  # Nếu không có giao dịch, support = 0.0
    sum_indexes = None
    for item in itemset:
        indexes = transaction_index.get(item)
        if indexes is None:
            return 0.0  # Nếu item không có trong danh sách giao dịch, support = 0.0
        if sum_indexes is None:
            sum_indexes = indexes
        else:
            sum_indexes = sum_indexes.intersection(indexes)
    # Tính support bằng số lượng giao dịch chứa itemset chia cho tổng số giao dịch.
    return float(len(sum_indexes)) / num_transaction


# Hàm để tạo ra các ứng viên tiếp theo cho thuật toán Apriori

In [66]:
def create_next_candidates(prev_candidates, length):
    # Tạo danh sách các mặt hàng đã được sắp xếp
    # fronzenset cũng giống như set nhưng nó immutable nên có thể sử dụng như là key của dictionary
    items = sorted(frozenset(chain.from_iterable(prev_candidates)))
    # Tạo danh sách các ứng viên tiếp theo bằng cách lấy tất cả các kết hợp có độ dài là "length"
    tmp_next_candidates = (frozenset(x) for x in combinations(items, length))
    # Nếu độ dài "length" nhỏ hơn 3 thì trả về toàn bộ ứng viên vì các tập con của nó cũng giống như tập items
    if length < 3:
        return list(tmp_next_candidates)
    # Tạo danh sách các ứng viên tiếp theo chỉ bao gồm các ứng viên có kích thước là "length" và tất cả các tập con kích thước "length - 1" thuộc danh sách các ứng viên trước đó
    next_candidates = [
        candidate for candidate in tmp_next_candidates
        if all(
            frozenset(x) in prev_candidates
            for x in combinations(candidate, length - 1))
    ]
    # Trả về danh sách các ứng viên tiếp theo
    return next_candidates


# Hàm tạo các bộ hỗ trợ (support records) cho các tập itemset trong các giao dịch


In [67]:

def create_support_records(transaction_items, transaction_index, num_transaction, min_support):
    # Khởi tạo tập ứng viên đầu tiên chỉ chứa các item đơn lẻ.
    candidates = [frozenset([item]) for item in transaction_items]
    length = 1
    support_records=[] # Danh sách chứa các SupportRecord thỏa mãn điều kiện
    # Tiếp tục tạo các tập ứng viên mới cho đến khi không còn ứng viên nào thỏa mãn điều kiện.
    while candidates:
        relations = set()  # Tập hỗ trợ (support set) chứa các tập ứng viên thỏa mãn điều kiện.
        for relation_candidate in candidates:
            # Tính support của tập ứng viên hiện tại.
            support = calc_support(transaction_index, num_transaction, relation_candidate)
            if support < min_support:
                continue  # Nếu support nhỏ hơn min_support, bỏ qua.
            candidate_set = frozenset(relation_candidate)
            relations.add(candidate_set)
            support_records.append(SupportRecord(candidate_set,support))
        length += 1
        candidates = create_next_candidates(relations, length)  # Tạo các ứng viên mới có độ dài tăng dần.
    return support_records # Trả về danh sách các SupportRecord thỏa mãn điều kiện.

# Hàm tạo các thống kê được sắp xếp theo thứ tự của các phần tử trong itemsets

In [68]:

def create_ordered_statistics(transaction_index, num_transaction, record):
    items = record.items
    sorted_items = sorted(items) # sắp xếp lại các phần tử trong itemsets
    ordered_statistics = [] # danh sách các thống kê sắp xếp

    for base_length in range(len(items)): # duyệt qua các kích thước base_length
        for combination_set in combinations(sorted_items, base_length): # duyệt qua tất cả các tập con có kích thước base_length
            items_base = frozenset(combination_set) # tập hợp các phần tử trong base
            items_add = frozenset(items.difference(items_base)) # tập hợp các phần tử còn lại (không thuộc base)
            confidence = ( # tính toán confidence và lift
                record.support / calc_support(transaction_index, num_transaction, items_base))
            lift = confidence / calc_support(transaction_index, num_transaction, items_add)

            # thêm vào danh sách ordered_statistics một OrderedStatistic mới
            ordered_statistics.append(OrderedStatistic(
                frozenset(items_base), frozenset(items_add), confidence, lift))

    return ordered_statistics

# Hàm dùng để lọc các association rule theo ngưỡng min_confidence và min_lift cho trước.



In [69]:
def filter_ordered_statistics(ordered_statistics, min_confidence, min_lift):
    # danh sách để lưu trữ các association rule được giữ lại
    filter_ordered_statistics_list = []
    # duyệt qua từng association rule trong danh sách ordered_statistics
    for ordered_statistic in ordered_statistics:
        # nếu confidence hoặc lift của association rule đó nhỏ hơn ngưỡng tương ứng thì bỏ qua và không thêm vào filter_ordered_statistics_list
        if ordered_statistic.confidence < min_confidence:
            continue
        if ordered_statistic.lift < min_lift:
            continue
        # ngược lại, thêm association rule đó vào filter_ordered_statistics_list
        filter_ordered_statistics_list.append(ordered_statistic)
    # trả về filter_ordered_statistics_list sau khi lọc xong
    return filter_ordered_statistics_list


# Hàm thuật toán chính apriori

In [70]:
def my_apriori(transactions, min_support=0.1, min_confidence=0.0, min_lift=0.0):
    if min_support <= 0:
        raise ValueError('minimum support must be > 0')

    # tính toán các chỉ số cho các giao dịch, danh sách các mục trong các giao dịch và một bản đồ từ mục đến các giao dịch chứa chúng
    # num_transaction là số giao dịch trong transactions.
    # transaction_items là danh sách các item trong các giao dịch.
    # transaction_index là danh sách index của các item  trong các giao dịch chứa chúng
    num_transaction, transaction_items, transaction_index = calc_num_items_index(
        transactions)
    support_records = create_support_records(
        transaction_items, transaction_index, num_transaction, min_support)
    relation_records = [] # danh sách các relation records
    for support_record in support_records:
        ordered_statistics = list(
            filter_ordered_statistics(
                create_ordered_statistics(
                    transaction_index, num_transaction, support_record),
                min_confidence=min_confidence,
                min_lift=min_lift,
            )
        )
        if not ordered_statistics:
            continue
        relation_records.append(RelationRecord(
            support_record.items, support_record.support, ordered_statistics))
    return relation_records


# So sánh kết quả với hàm apriori

In [71]:
association_rules=my_apriori(records,min_support=0.4,min_confidence=0.7,min_lift=1.2)

In [72]:
association_results=list(association_rules)
association_results

[RelationRecord(items=frozenset({'Apple', 'Wine', 'Bread'}), support=0.45454545454545453, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Apple', 'Wine'}), items_add=frozenset({'Bread'}), confidence=0.9090909090909091, lift=1.25)]),
 RelationRecord(items=frozenset({'Apple', 'Milk', 'Butter'}), support=0.4090909090909091, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Apple', 'Milk'}), items_add=frozenset({'Butter'}), confidence=0.8181818181818182, lift=1.2000000000000002)]),
 RelationRecord(items=frozenset({'Butter', 'Milk', 'Bread'}), support=0.5, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Butter'}), items_add=frozenset({'Milk', 'Bread'}), confidence=0.7333333333333334, lift=1.241025641025641), OrderedStatistic(items_base=frozenset({'Milk', 'Bread'}), items_add=frozenset({'Butter'}), confidence=0.8461538461538461, lift=1.241025641025641)]),
 RelationRecord(items=frozenset({'Butter', 'Wine', 'Bread'}), support=0.45454545454545453, ordered_stati

In [73]:
output_DataFrame=pd.DataFrame(edited_inspect(association_results),columns=['Left_Hand_Side','Right_Hand_Side','Support','Confidence','Lift'])
output_DataFrame

,Left_Hand_Side,Right_Hand_Side,Support,Confidence,Lift
0,"(Apple, Wine)",(Bread),0.454545,0.909091,1.250000
1,"(Apple, Milk)",(Butter),0.409091,0.818182,1.200000
2,(Butter),"(Milk, Bread)",0.500000,0.733333,1.241026
3,"(Milk, Bread)",(Butter),0.500000,0.846154,1.241026
4,"(Wine, Butter)",(Bread),0.454545,0.909091,1.250000
5,"(Wine, Butter)","(Milk, Bread)",0.409091,0.818182,1.384615
6,"(Milk, Wine, Bread)",(Butter),0.409091,0.818182,1.200000
7,"(Milk, Wine, Butter)",(Bread),0.409091,0.900000,1.237500


Ta thấy kết quả giống với quả của hàm apriori